In [1]:
import os
import openai
import json
from Keys import openai_keys
import re


openai.organization = openai_keys['organization']
openai.api_key = openai_keys['api_key']
openai.File.create(
  file=open("mydata.jsonl", "rb"),
  purpose='fine-tune'
)

FileNotFoundError: [Errno 2] No such file or directory: 'mydata.jsonl'

In [8]:
from IPython.display import Markdown, display

prompts = [{"role": "system", "content":
    'You are a software development specialist'}]

In [13]:
prompt = input()
prompts.append({"role": "user", "content": prompt})
chat = openai.ChatCompletion.create(model="gpt-4", messages=prompts)
response = (chat.choices[0].message.content)
display(Markdown(response))
prompts.append({"role": "assistant", "content": response})

To get the subtitles (also known as closed captions) for a YouTube video, `captions().list` method provided by Youtube Data API v3 can be used. 

For more details, check the official YouTube API documentation: https://developers.google.com/youtube/v3/docs/captions

However, please note that not all videos have available closed captions, and some may have multiple sets of captions in different languages. You can specify the language of the subtitle that you want.

But it's important to note that downloading YouTube captions requires proper authorization with OAuth 2.0 using the 'https://www.googleapis.com/auth/youtube.force-ssl' scope.

Here is a simple demonstration code. But remember, for proper OAuth 2.0 setup and usage, please refer to the official Google OAuth 2.0 guide:

```python
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.credentials import Credentials
import os
import io

# Load credentials from the 'token.json' file
creds = Credentials.from_authorized_user_file('token.json')

youtube = build('youtube', 'v3', credentials=creds)

def get_captions(youtube, video_id):
    try:
        # Retrieve caption tracks list
        captions = youtube.captions().list(
            part="snippet",
            videoId=video_id
        ).execute()

        # Select the first caption track
        caption = captions['items'][0]

        # Download the caption track in the language you want
        subtitle = youtube.captions().download(
            id=caption['id'],
            tfmt='srt' # VTT or SRT subtitle formats.
        ).execute()

        # Save the subtitle to a .srt (or .vtt) file
        with open("subtitle.srt", "w", encoding="utf-8") as f:
            f.write(subtitle)

        print("Subtitle has been written to subtitle.srt")

    except HttpError as e:
        print(f"A HTTP error {e.resp.status} occurred:\n{e.content}")

get_captions(youtube, 'VIDEO_ID') # place your video id here
```

In the code snippet above:

1. We are loading the user credentials from 'token.json'. You should replace this with the path to your OAuth 2.0 credentials file. You can generate this file using Google's OAuth 2.0 Playground and save it to your script's directory.

2. You need to replace "VIDEO_ID" with your actual video ID.

Please keep in mind that reading the captions of a Youtube video with the API costs 200 quota units, and writing a simple list operation costs 50. Always be mindful of Google's daily quota limits when using their API services.

In [11]:
display(Markdown(prompts[2]['content']))

To get the IDs of all Youtube videos from a specific channel, you can use the YouTube API with the Google API client for Python. Here's a basic script to get you started:

```python
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

api_key = 'YOUR_API_KEY' # replace with your own API key
youtube = build('youtube', 'v3', developerKey=api_key)

def get_channel_videos(channel_id):
    res = youtube.channels().list(id=channel_id, part='contentDetails').execute()
    playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    videos = []
    next_page_token = None

    while 1:
        res = youtube.playlistItems().list(playlistId=playlist_id, 
                                           part='snippet', 
                                           maxResults=50,
                                           pageToken=next_page_token).execute()
        videos += res['items']
        next_page_token = res.get('nextPageToken')

        if next_page_token is None:
            break

    return videos

videos = get_channel_videos('CHANNEL_ID') # replace with the channel id 
video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))

print("\n".join(video_ids))
```

This script will retrieve all the videos from the specified channel's Uploads playlist, then extract the video ID from each video in that playlist.

Important points to note:
- Replace `'YOUR_API_KEY'` with your actual API key that you obtained from the Google Developer Console.
- Replace `'CHANNEL_ID'` with the ID of the YouTube Channel from which you want to fetch all video IDs.
- API quotas mean you can only make a certain number of requests per day - if you are exceeding your quota, you may need to wait, or request additional quota.
- Depending on the number of videos, fetching all video IDs might take some time. If request times out or you encounter a similar problem, consider fetching in batches or using pagination.
